In [2]:
#optional installations: 
#!pip install yfinance --upgrade --no-cache-dir
#!pip3 install pandas_datareader


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# ___library_import_statements___
import pandas as pd

# for pandas_datareader, otherwise it might have issues, sometimes there is some version mismatch
pd.core.common.is_list_like = pd.api.types.is_list_like

# make pandas to print dataframes nicely
pd.set_option('expand_frame_repr', False)  

import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time

#newest yahoo API 
import yfinance as yahoo_finance

#optional 
#yahoo_finance.pdr_override()

%matplotlib inline

In [112]:
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi


def stochastics( dataframe, low, high, close, k, d ):
    """
    Fast stochastic calculation
    %K = (Current Close - Lowest Low)/
    (Highest High - Lowest Low) * 100
    %D = 3-day SMA of %K

    Slow stochastic calculation
    %K = %D of fast stochastic
    %D = 3-day SMA of %K

    When %K crosses above %D, buy signal 
    When the %K crosses below %D, sell signal
    """

    df = dataframe.copy()

    # Set minimum low and maximum high of the k stoch
    low_min  = df[low].rolling( window = k ).min()
    high_max = df[high].rolling( window = k ).max()

    # Fast Stochastic
    df['k_fast'] = 100 * (df[close] - low_min)/(high_max - low_min)
    df['d_fast'] = df['k_fast'].rolling(window = d).mean()

    # Slow Stochastic
    df['k_slow'] = df["d_fast"]
    df['d_slow'] = df['k_slow'].rolling(window = d).mean()
    df['d_div'] = df['d_fast'] - df['d_slow']

    return df

def MACD(dataframe,close,span1,span2,sig_span):
    df = dataframe.copy()
    exp1 = df[close].ewm(span=span1, adjust=False).mean()
    exp2 = df[close].ewm(span=span2, adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=sig_span, adjust=False).mean()
    df['MACD line'] = macd
    df['Sig Line'] = exp3
    df['MACD_Div'] = macd-exp3
#     plt.plot(df.Date, macd, label='MACD', color = '#EBD2BE')
#     plt.plot(df.Date, exp3, label='Signal Line', color='#E5A4CB')
#     plt.legend(loc='upper left')
#     plt.show()
    return df

In [120]:
# ___variables___
ticker = 'XLF'

start_time = datetime.datetime(2017, 10, 1)
#end_time = datetime.datetime(2019, 1, 20)
end_time = datetime.datetime.now().date().isoformat() 
start_time = (datetime.datetime.now().date() - datetime.timedelta(days = 90)).isoformat()
print("Start Date: ", start_time)
print("End Date: ", end_time )
# yahoo gives only daily historical data
connected = False
while not connected:
    try:
        ticker_df = web.get_data_yahoo(ticker, start=start_time, end=end_time)
        connected = True
        print('connected to yahoo')
    except Exception as e:
        print("type error: " + str(e))
        time.sleep( 5 )
        pass   

# use numerical integer index instead of date    
ticker_df = ticker_df.reset_index()
print(ticker_df.head(5))
df = ticker_df

Start Date:  2020-11-24
End Date:  2021-02-22
connected to yahoo
        Date       High        Low       Open      Close      Volume  Adj Close
0 2020-11-24  28.690001  28.100000  28.170000  28.660000  60983200.0  28.507093
1 2020-11-25  28.580000  28.219999  28.500000  28.559999  45204600.0  28.407625
2 2020-11-27  28.590000  28.360001  28.559999  28.469999  26981200.0  28.318106
3 2020-11-30  28.379999  27.830000  28.299999  27.879999  72524200.0  27.731253
4 2020-12-01  28.570000  28.290001  28.400000  28.309999  53977300.0  28.158960


In [121]:
df['RSI'] = computeRSI(df['Adj Close'], 14)
df = stochastics( df, 'Low', 'High', 'Close', 14, 3 )
df = MACD(df,'Close',12,26,9)
# print(df)

In [122]:
day0 = df['Date'].iloc[-1]
day7 = df['Date'].iloc[-7]
day14 = df['Date'].iloc[-14]

#RSI values: 
AllRSI = np.array([df[df['Date']==day0]['RSI'].values,df[df['Date']==day7]['RSI'].values,df[df['Date']==day14]['RSI'].values])
#d_slow values: 
AllSS = np.array([df[df['Date']==day0]['d_slow'].values,df[df['Date']==day7]['d_slow'].values,df[df['Date']==day14]['d_slow'].values])
#d_fast values
AllSF = np.array([df[df['Date']==day0]['d_fast'].values,df[df['Date']==day7]['d_fast'].values,df[df['Date']==day14]['d_fast'].values])
#d_div values
AllSD = np.array([df[df['Date']==day0]['d_div'].values,df[df['Date']==day7]['d_div'].values,df[df['Date']==day14]['d_div'].values])
#MACD_Div values
AllMD = np.array([df[df['Date']==day0]['MACD_Div'].values,df[df['Date']==day7]['MACD_Div'].values,df[df['Date']==day14]['MACD_Div'].values])

row = df[df['Date']==end_time]['d_slow'].values
print(df['Date'].iloc[-1])
print(AllRSI, AllSS, AllSF, AllMD)

#Algorithm for entry ot exit
CTI = [] #CUmulative Trade Indicator

#RSI Score
if AllRSI[1]/AllRSI[2] > 1:
    Mult=2.0
else: 
    Mult=1.0

if AllRSI[0] > 20 and AllRSI[0] <= 50:
    CTI.append(1.0*Mult)
elif AllRSI[0] > 50 and AllRSI[0]<80:
    CTI.append(-1.0*Mult)
else: 
    CTI.append(0.0*Mult)

    
#d_slow Score
if AllSS[1]/AllSS[2] > 1:
    Mult = 2.0
else: 
    Mult = 1.0

if AllSS[0]>25 and AllSS[0]<35:
    CTI.append(2.0*Mult)
elif AllSS[0]>=35 and AllSS[0]<55:
    CTI.append(1.0*Mult)
elif AllSS[0]>=55 and AllSS[0]<75:
    CTI.append(0.0*Mult)
elif AllSS[0]>=75 and AllSS[0]<85:
    CTI.append(-1.0*Mult)
elif AllSS[0]>=85:
    CTI.append(-2.0*Mult)
else: 
    CTI.append(0.0*Mult)
    

#MACD_Div
if AllMD[0] > 0:
    Mult = 2.0
else: 
    Mult = 1.0

if AllMD[1]/AllMD[2]>1 and AllMD[1]>0:
    CTI.append(2.0*Mult)
elif AllMD[1]/AllMD[2]<1 and AllMD[1]>0:
    CTI.append(1.0*Mult)
elif AllMD[1]/AllMD[2]>1 and AllMD[1]<0:
    CTI.append(-2.0*Mult)
elif AllMD[1]/AllMD[2]<1 and AllMD[1]<0:
    CTI.append(-1.0*Mult)
else: 
    CTI.append(0.0*Mult)
    
safeCTI = sum(CTI)/10.0

#d_div
if AllSD[0] > 0:
    Mult = 2.0
else: 
    Mult = 1.0

if AllSD[1]/AllSD[2]>1 and AllSD[1]>0:
    CTI.append(2.0*Mult)
elif AllSD[1]/AllSD[2]<1 and AllSD[1]>0:
    CTI.append(1.0*Mult)
elif AllSD[1]/AllSD[2]>1 and AllSD[1]<0:
    CTI.append(-2.0*Mult)
elif AllSD[1]/AllSD[2]<1 and AllSD[1]<0:
    CTI.append(-1.0*Mult)
else: 
    CTI.append(0.0*Mult)
        
EECTI = sum(CTI)/14.0


print("Safe CTI: ",safeCTI)
print("Early Entry CTI: ",EECTI)

# if CTI > 0.5: 
#     print("Buy")
# elif CTI > 0.8:
#     print("Strong Buy")
# elif CTI < -0.5: 
#     print("Sell")
# elif CTI < -0.8:
#     print("Strong Sell")
# else: 
#     print("No Trade")

2021-02-22 00:00:00
[[71.9292947 ]
 [61.893733  ]
 [52.28599471]] [[96.26713759]
 [94.44049429]
 [15.660404  ]] [[96.55643681]
 [92.4891006 ]
 [22.14287807]] [[ 0.16332703]
 [ 0.10531696]
 [-0.19410529]]
Safe CTI:  -0.4
Early Entry CTI:  -0.42857142857142855
